# Probability and fitting tuning curves

1. Oriented gratings
2. Spikes as poisson process
    * counting process
    * rate
    * count probability as a function of rate
3. Conditional probability
    * rate conditional on time
    * rate conditional on stimulus
4. Receptive field
    * tuning location in space
    * tuning for orientation

In [1]:
import os
import sys
sys.path.append('..')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache


/Users/drewheadley/anaconda3/envs/decode_brain/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# initialize data cache
allen_dir = ['..','data','spike_allen']
manifest_path = os.path.join(*allen_dir,'manifest.json')
cache = EcephysProjectCache(manifest=manifest_path)

# get session
sess_id = 750332458
sess = cache.get_session_data(sess_id, filter_by_validity=False)

/Users/drewheadley/anaconda3/envs/decode_brain/lib/python3.9/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/Users/drewheadley/anaconda3/envs/decode_brain/lib/python3.9/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached namespace 'core' version 2.2.2 because version 2.6.0-alpha is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


In [4]:
stims = sess.stimulus_presentations
stims['stimulus_name'].unique()

/Users/drewheadley/anaconda3/envs/decode_brain/lib/python3.9/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/Users/drewheadley/anaconda3/envs/decode_brain/lib/python3.9/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached namespace 'core' version 2.2.2 because version 2.6.0-alpha is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


array(['spontaneous', 'gabors', 'flashes', 'drifting_gratings',
       'natural_movie_three', 'natural_movie_one', 'static_gratings',
       'natural_scenes'], dtype=object)

In [11]:
dg_stim = stims[stims['stimulus_name']=='drifting_gratings']

# list stimulus properties
spat_freq = dg_stim['spatial_frequency'].unique()
print('Spatial frequencies: {}'.format(spat_freq))

temp_freq = dg_stim['temporal_frequency'].unique()
print('Temporal frequencies: {}'.format(temp_freq))

orient = dg_stim['orientation'].unique()
print('Orientations: {}'.format(orient))

Spatial frequencies: ['0.04' 'null']
Temporal frequencies: [4.0 2.0 15.0 8.0 1.0 'null']
Orientations: [180.0 225.0 135.0 0.0 315.0 90.0 270.0 'null' 45.0]


In [14]:
len(dg_stim[(dg_stim['temporal_frequency']==1.0)&(dg_stim['orientation']==180.0)])

15

In [17]:
stim_starts = dg_stim.groupby(['orientation', 'temporal_frequency'])['start_time']

In [10]:
dg_stim

,stimulus_block,start_time,stop_time,spatial_frequency,orientation,x_position,contrast,size,temporal_frequency,phase,y_position,stimulus_name,color,frame,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,,,,,,,
3798,2.0,1585.647748,1587.649398,0.04,180.0,null,0.8,"[250.0, 250.0]",4.0,"[21235.93333333, 21235.93333333]",null,drifting_gratings,null,null,2.00165,246
3799,2.0,1588.650242,1590.651902,0.04,225.0,null,0.8,"[250.0, 250.0]",4.0,"[21235.93333333, 21235.93333333]",null,drifting_gratings,null,null,2.00166,247
3800,2.0,1591.652728,1593.654418,0.04,135.0,null,0.8,"[250.0, 250.0]",2.0,"[21235.93333333, 21235.93333333]",null,drifting_gratings,null,null,2.00169,248
3801,2.0,1594.655252,1596.656912,0.04,0.0,null,0.8,"[250.0, 250.0]",2.0,"[21235.93333333, 21235.93333333]",null,drifting_gratings,null,null,2.00166,249
3802,2.0,1597.657758,1599.659418,0.04,315.0,null,0.8,"[250.0, 250.0]",15.0,"[21235.93333333, 21235.93333333]",null,drifting_gratings,null,null,2.00166,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49428,7.0,5383.820662,5385.822322,0.04,135.0,null,0.8,"[250.0, 250.0]",15.0,"[21235.93333333, 21235.93333333]",null,drifting_gratings,null,null,2.00166,254
49429,7.0,5386.823178,5388.824828,0.04,0.0,null,0.8,"[250.0, 250.0]",1.0,"[21235.93333333, 21235.93333333]",null,drifting_gratings,null,null,2.00165,261
49430,7.0,5389.825682,5391.827362,0.04,315.0,null,0.8,"[250.0, 250.0]",15.0,"[21235.93333333, 21235.93333333]",null,drifting_gratings,null,null,2.00168,250
